<a href="https://colab.research.google.com/github/DantonCoote/My_RRC_HUB/blob/NNDL_Project/First_Iteration_BART_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoTokenizer, BartForConditionalGeneration
# Loading our pretrained model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# tokenizer is simply a system that encodes the text data into numerical
# information that is then run through the model

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import pandas as pd

#this is a random kaggle dataset I found that has stock summaries
data = pd.read_csv('news_stock_price.csv')
data.head()

,date,url,full_text,summary,voo_close,voo_volume,qqq_close,qqq_volume,dia_close,dia_volume
0,2022-10-08,https://www.livemint.com/market/stock-market-n...,Have you read your account statements for the ...,And it’s natural to avoid looking too closely ...,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-08,https://www.zawya.com/en/markets/equities/wall...,Wall Street fell sharply on Friday following a...,"Nonfarm payrolls rose by 263,000 jobs, more th...",NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-08,https://www.goodisonnews.com/2022/10/08/stock-...,"By Matthew Shaw 8th Oct, 2022 | 8:10am\n\nStoc...","By Matthew Shaw 8th Oct, 2022 | 8:10amStock ma...",NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-08,https://www.livemint.com/market/stock-market-n...,Stock split 2022: The board of directors of An...,Stock split 2022: The board of directors of An...,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-08,https://finance.yahoo.com/news/crypto-exchange...,(Reuters) - Cryptocurrency exchange Huobi Glob...,(Reuters) - Cryptocurrency exchange Huobi Glob...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# removing all irrelevent information
data = data.drop(columns=['date', 'url',
                          'voo_close','voo_volume',
                          'qqq_close', 'qqq_volume',
                          'dia_close','dia_volume'])
data.head()# checking what is left

,full_text,summary
0,Have you read your account statements for the ...,And it’s natural to avoid looking too closely ...
1,Wall Street fell sharply on Friday following a...,"Nonfarm payrolls rose by 263,000 jobs, more th..."
2,"By Matthew Shaw 8th Oct, 2022 | 8:10am\n\nStoc...","By Matthew Shaw 8th Oct, 2022 | 8:10amStock ma..."
3,Stock split 2022: The board of directors of An...,Stock split 2022: The board of directors of An...
4,(Reuters) - Cryptocurrency exchange Huobi Glob...,(Reuters) - Cryptocurrency exchange Huobi Glob...


In [6]:
data.info()# ensuring cleanliness

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  4699 non-null   object
 1   summary    4699 non-null   object
dtypes: object(2)
memory usage: 73.5+ KB


In [7]:
# actually testing how effective BART is by using a single case
test1 = data.loc[0,'full_text']
tstSum = data.loc[0,'summary']
test1

'Have you read your account statements for the just-finished third quarter?\n\nIf you’re anything like me, your answer is no. You probably already know stocks are down 20% and bonds 14% for the year to date.\n\nLooking at our losses won’t make them any smaller. But it might make us feel smaller. And it’s natural to avoid looking too closely at any evidence that might undermine our belief that we are skilled investors. In down markets, however, making good decisions often requires admitting things about ourselves we would much rather ignore.\n\nLet’s start by recognizing that inertia can be a choice.\n\nSince the end of March, when stocks were within a whisker of their all-time highs, investors have withdrawn about $80 billion from stock mutual funds and exchange-traded funds, according to the Investment Company Institute.\n\nU.S. and international stock funds held $19.3 trillion at the end of March. So, even though inflation can’t seem to fall and stocks can’t seem to rise, investors h

In [15]:
import operator as op
# counting individual words, needs more fine tuning
def word_count(text):
    return op.countOf(text, ' ' or '. ') + 1

In [16]:
word_count(test1)

917

In [10]:
test1 = test1.replace('\n', ' ')
test1# removing newlines from text and checking what is left

'Have you read your account statements for the just-finished third quarter?  If you’re anything like me, your answer is no. You probably already know stocks are down 20% and bonds 14% for the year to date.  Looking at our losses won’t make them any smaller. But it might make us feel smaller. And it’s natural to avoid looking too closely at any evidence that might undermine our belief that we are skilled investors. In down markets, however, making good decisions often requires admitting things about ourselves we would much rather ignore.  Let’s start by recognizing that inertia can be a choice.  Since the end of March, when stocks were within a whisker of their all-time highs, investors have withdrawn about $80 billion from stock mutual funds and exchange-traded funds, according to the Investment Company Institute.  U.S. and international stock funds held $19.3 trillion at the end of March. So, even though inflation can’t seem to fall and stocks can’t seem to rise, investors have taken 

In [11]:
inputs = tokenizer(test1, return_tensors="pt", max_length=1024, truncation=True)

In [19]:
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=300, max_length=600)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Investors have withdrawn about $80 billion from stock mutual funds and exchange-traded funds. Even though inflation can’t seem to fall, investors have taken only 0.4% of their money out of stock funds. New research shows that fund managers lose an average of about 0.8 percentage points of return annually through poor selling decisions. If what you sold is outperforming what you hold, you’ve been selling the wrong investments, says Annie Duke, a cognitive champion and author of “Quit: When to Walk Away"  Let’s say you bought bitcoin last year because you believed it was a protective hedge against inflation. Putting you in a position to kill inflation would have committed you to a place in which you would have to commit to killing inflation in the first place. It would also kill you if you bought an investment to meet a set of conditions, such as that it would meet a certain level of returns. It’d kill you to commit you to something that would kill you in the long run. It could kill you